## CUDA python implementation of [py_lucas_kanade](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html) example

In [8]:
import numpy as np
import os
import cv2
import time
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

vidPath = os.environ['OPENCV_TEST_DATA_PATH'] + '/cv/video/768x576.avi'

## Original CPU implementation

In [10]:
cap = cv2.VideoCapture(vidPath)

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
etime = 0
while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    t = time.perf_counter()
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    etime += (time.perf_counter() - t)
    
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.1.1) E:\Dev\Repos\OpenCV\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [11]:
etime

0.16676009999997632

## Naive CUDA implementation without pre-alloc, streams or other optimizations

In [15]:
cap = cv2.VideoCapture(vidPath)

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray_device = cv2.cuda_GpuMat(cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY))
detector_device =  cv2.cuda.createGoodFeaturesToTrackDetector(cv2.CV_8UC1, feature_params['maxCorners'], \
                                           feature_params['qualityLevel'], feature_params['minDistance'], \
                                           feature_params['blockSize'])
p0_device = detector_device.detect(old_gray_device)

optFlow = cv2.cuda_SparsePyrLKOpticalFlow.create()

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
frame_gray_device = cv2.cuda_GpuMat()
p0 = p0_device.download()
etime = 0
while(1):
    ret,frame = cap.read()
    frame_gray_device.upload( cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))    

    # calculate optical flow
    t = time.perf_counter()
    p1_device, st_device, err = optFlow.calc(old_gray_device,frame_gray_device,p0_device,None)
    etime += (time.perf_counter() - t)
    
    # dload points
    p1 = p1_device.download()
    st = st_device.download()
        
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    frame_gray_device.copyTo(old_gray_device)
    p0 = np.expand_dims(good_new,axis=0)
    p0_device.upload(p0)

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.1.1) E:\Dev\Repos\OpenCV\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [17]:
etime

0.10651610000002165